In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # MR
# mr_trainpt_test_dfead_csv("Mahapt_test_dfain.csv"pt_test_df_df   = pd.read_csv("Mpt_test_df_Val.csv")
# mr_test_df  = pd.read_cpt_test_dfnpt_test_dfcsvpt_test_dftest_dfrain_df = pd.read_csv("MahaSent_ST_Train.csv")
# st_val_df   = pd.read_csv("Mahapt_test_dfpt_test_dfst_test_df  = pd.read_csv("MahaSent_ST_Test.csv")

# # GT
# pt_train_df = pd.read_csv("tweets-train_GT.csv")
# pt_val_df   = pd.read_csv("tweets-valid_GT.csv")
# pt_test_df  = pd.read_csv("tweets-test_GT.csv")

mr_train_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_MR/MahaSent_MR_Train.csv")
mr_val_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_MR/MahaSent_MR_Val.csv")
mr_test_df = pd.read_csv('/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_MR/MahaSent_MR_Test.csv')

pt_train_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_PT/tweets-train.csv")
pt_val_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_PT/tweets-valid.csv")
pt_test_df = pd.read_csv('/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_PT/tweets-test.csv')

gt_train_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_GT/tweets-train.csv")
gt_val_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_GT/tweets-valid.csv")
gt_test_df = pd.read_csv('/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_GT/tweets-test.csv')

st_train_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Train.csv")
st_val_df   = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Val.csv")
st_test_df  = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Test.csv")

In [ ]:
print("MR Train:")
print(mr_train_df.head())
print("\nPT Train:")
print(pt_train_df.head())
print("\nGT Train:")
print(gt_train_df.head())

MR Train:
   Unnamed: 0                                   marathi_sentence  label
0           0  माने यांचा घटस्फोट झाला आहे तर मोहितेने नुकतेच...     -1
1           1  एका रात्रीत घडणारी किंबहुना बिघडणारी ही गोष्ट आहे     -1
2           2  जरी आघात समजण्यायोग्य आहे जरी चित्रपटाला खराब ...     -1
3           3  पण तो असा आघातही अनुभवत आहे की तो कोणाशीही शेअ...     -1
4           4               छोटे-छोटे गैरसमज मोठ्या अडचणीत येतात     -1

PT Train:
                                               tweet  label
0  ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे व...     -1
1  सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिस...     -1
2  उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांच...     -1
3  आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भू...      1
4  बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...     -1

GT Train:
                                               tweet  label  political
0          होता होता राहीलेला  निवडणूक मारो मर्ज़ीभई     -1          0
1                         खरा लखोब

In [ ]:
# RENAME COLUMNS → unify to 'text'
mr_train_df = mr_train_df.rename(columns={"marathi_sentence": "text"})
gt_train_df = gt_train_df.rename(columns={"tweet": "text"})
pt_train_df = pt_train_df.rename(columns={"tweet": "text"})

mr_train_df = mr_train_df[["text", "label"]]
gt_train_df = gt_train_df[["text", "label"]]
pt_train_df = pt_train_df[["text", "label"]]

In [ ]:
# MERGE TRAIN: MR + GT + ST
train_df = pd.concat([mr_train_df, pt_train_df, gt_train_df], ignore_index=True)
train_df = train_df.sample(frac=1, random_state=42)

In [ ]:
# VALIDATION MERGE
mr_val_df = mr_val_df.rename(columns={"marathi_sentence": "text"})
gt_val_df = gt_val_df.rename(columns={"tweet": "text"})
pt_val_df = pt_val_df.rename(columns={"tweet": "text"})

val_df = pd.concat([
    mr_val_df[["text", "label"]],
    pt_val_df[["text", "label"]],
    gt_val_df[["text", "label"]],
], ignore_index=True)

val_df = val_df.sample(frac=1, random_state=42)

In [ ]:
# TEST MERGE
mr_test_df = mr_test_df.rename(columns={"marathi_sentence": "text"})
gt_test_df = gt_test_df.rename(columns={"tweet": "text"})
pt_test_df = pt_test_df.rename(columns={"tweet": "text"})

test_df = pd.concat([
    mr_test_df[["text", "label"]],
    pt_test_df[["text", "label"]],
    gt_test_df[["text", "label"]],
], ignore_index=True)

test_df = test_df.sample(frac=1, random_state=42)

In [ ]:
# CLEAN LABELS + TEXT
for df in [train_df, val_df, test_df]:
    df["label"] = df["label"].astype(int) + 1   # -1→0, 0→1, 1→2
    df["text"] = df["text"].astype(str).fillna("")

In [ ]:
# Convert to HF Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)

In [ ]:
model_name = "l3cube-pune/marathi-bert-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
def tokenize_fn(batch):
    texts = [str(x) for x in batch["text"]]
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_encoded = train_dataset.map(tokenize_fn, batched=True)
val_encoded   = val_dataset.map(tokenize_fn, batched=True)
test_encoded  = test_dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/36114 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro": recall_score(labels, preds, average="macro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./mr_gt_st_marathi_bert_v2",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     logging_steps=50,
#     fp16=True,
#     report_to="none"
# )
training_args = TrainingArguments(
    output_dir="./results",                   # 📁 Where to save checkpoints
    eval_strategy="epoch",              # 🧪 Evaluate after each epoch
    save_strategy="epoch",                    # 💾 Save model after each epoch
    per_device_train_batch_size=64,            # 🧠 Reduce memory usage
    per_device_eval_batch_size=64,             # ⚖️ Matching batch size for eval
    num_train_epochs=2,                       # 🔁 You can try 1 or 2 for speed
    learning_rate=2e-5,                       # 🚀 Good default for BERT
    # warmup_steps=100,                         # 🔥 Warmup helps training stability
    weight_decay=0.01,                        # 📉 Helps avoid overfitting
    logging_dir="./logs",                     # 📂 Logs directory
    logging_steps=50,                         # 📝 Log every 50 steps
    load_best_model_at_end=True,              # 🏆 Automatically load best model
    metric_for_best_model="accuracy",         # 🎯 Use accuracy to pick best
    save_total_limit=1,                       # 💾 Keep only last best model
    fp16=True,                                # ⚡ Mixed precision for faster training
    report_to="none",                         # 📊 Turn off W&B if you’re not using it
    seed=42,         # Standard seed, might be what they used
    data_seed=42,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3780730124.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro
1,0.628100,0.568582,0.805556,0.805243,0.805556,0.805007
2,0.498500,0.510240,0.809556,0.808560,0.809556,0.808828


TrainOutput(global_step=1130, training_loss=0.6472388039648005, metrics={'train_runtime': 547.5809, 'train_samples_per_second': 131.904, 'train_steps_per_second': 2.064, 'total_flos': 4751038983914496.0, 'train_loss': 0.6472388039648005, 'epoch': 2.0})

In [ ]:
test_metrics = trainer.evaluate(test_encoded)
print("Combined MR + PT + GT Test metrics:", test_metrics)

Combined MR + PT + GT Test metrics: {'eval_loss': 0.5137449502944946, 'eval_accuracy': 0.812952380952381, 'eval_precision_macro': 0.8118581494635048, 'eval_recall_macro': 0.8129523809523809, 'eval_f1_macro': 0.8119349585149308, 'eval_runtime': 9.5838, 'eval_samples_per_second': 547.801, 'eval_steps_per_second': 8.66, 'epoch': 2.0}


In [ ]:
# ----- MR -----
mr_test_df_processed = mr_test_df.copy()
mr_test_df_processed['label'] = mr_test_df_processed['label'].astype(int) + 1
mr_test_df_processed['text'] = mr_test_df_processed['text'].astype(str).fillna('')
mr_test_dataset = Dataset.from_pandas(mr_test_df_processed)
mr_test_encoded = mr_test_dataset.map(tokenize_fn, batched=True)

mr_test_metrics = trainer.evaluate(mr_test_encoded)
print("MahaSent_MR Test metrics:", mr_test_metrics)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MahaSent_MR Test metrics: {'eval_loss': 0.5532221794128418, 'eval_accuracy': 0.7986666666666666, 'eval_precision_macro': 0.7977477894217454, 'eval_recall_macro': 0.7986666666666666, 'eval_f1_macro': 0.7968066236840453, 'eval_runtime': 2.6511, 'eval_samples_per_second': 565.796, 'eval_steps_per_second': 9.053, 'epoch': 2.0}


In [ ]:
#---------GT-----------
gt_test_df_processed = gt_test_df.copy()
gt_test_df_processed["label"] = gt_test_df_processed["label"].astype(int) + 1
gt_test_df_processed["text"] = gt_test_df_processed["text"].astype(str).fillna("")
gt_test_dataset = Dataset.from_pandas(gt_test_df_processed)
gt_test_encoded = gt_test_dataset.map(tokenize_fn, batched=True)
gt_test_metrics = trainer.evaluate(gt_test_encoded)
print("GT Test metrics:", gt_test_metrics)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

GT Test metrics: {'eval_loss': 0.5608096718788147, 'eval_accuracy': 0.7806666666666666, 'eval_precision_macro': 0.7813857530456941, 'eval_recall_macro': 0.7806666666666667, 'eval_f1_macro': 0.7809564137154226, 'eval_runtime': 4.5132, 'eval_samples_per_second': 332.361, 'eval_steps_per_second': 5.318, 'epoch': 2.0}


In [ ]:
# ----- PT -----
pt_test_df_processed = pt_test_df.copy()
pt_test_df_processed['label'] = pt_test_df_processed['label'].astype(int) + 1
pt_test_df_processed['text'] = pt_test_df_processed['text'].astype(str).fillna('')
pt_test_dataset = Dataset.from_pandas(pt_test_df_processed)
pt_test_encoded = pt_test_dataset.map(tokenize_fn, batched=True)

pt_test_metrics = trainer.evaluate(pt_test_encoded)
print("MahaSent_PT Test metrics:", pt_test_metrics)

Map:   0%|          | 0/2250 [00:00<?, ? examples/s]

MahaSent_PT Test metrics: {'eval_loss': 0.4560532867908478, 'eval_accuracy': 0.844, 'eval_precision_macro': 0.8445554389446782, 'eval_recall_macro': 0.844, 'eval_f1_macro': 0.843519249431501, 'eval_runtime': 3.9305, 'eval_samples_per_second': 572.449, 'eval_steps_per_second': 9.159, 'epoch': 2.0}


In [ ]:
# uploaded = files.upload()

Saving tweets-test_PT.csv to tweets-test_PT.csv


In [ ]:
st_test_df_processed = st_test_df.copy()
st_test_df_processed = st_test_df_processed.rename(columns={"marathi_text": "text"})
st_test_df_processed["label"] = st_test_df_processed["label"].astype(int) + 1
st_test_df_processed["text"] = st_test_df_processed["text"].astype(str).fillna("")

st_test_dataset = Dataset.from_pandas(st_test_df_processed)
st_test_encoded = st_test_dataset.map(tokenize_fn, batched=True)

st_test_metrics = trainer.evaluate(st_test_encoded)
print("MahaSent_ST Test metrics:", st_test_metrics)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MahaSent_ST Test metrics: {'eval_loss': 0.6440359950065613, 'eval_accuracy': 0.7326666666666667, 'eval_precision_macro': 0.7651662148244434, 'eval_recall_macro': 0.7326666666666667, 'eval_f1_macro': 0.7368857121392618, 'eval_runtime': 3.8867, 'eval_samples_per_second': 385.932, 'eval_steps_per_second': 6.175, 'epoch': 2.0}


In [ ]:
st_train_df.head()

,Unnamed: 0,marathi_text,label
0,0,मी तो मूर्ख आहे.,-1
1,1,आईने आत्महत्या केली आणि माझे वडील पळून गेले.,-1
2,2,- त्यांनी फक्त जॉयला मारले?,-1
3,3,मला माफ करा मी आज तिथेच केले नाही.,-1
4,4,"नाही, नक्कीच नाही.",-1
